# take2

**GOAL**: Keep It Stupid Simple = KISS.

**Dependencies:**
 - [pocketknives](https://github.com/zjwilliams20/pocketknives)

**References:**
 1. [ilqgames/python](https://github.com/HJReachability/ilqgames/blob/master/python)

In [ ]:
from time import perf_counter as pc
import warnings

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from decentralized import split_agents, plot_solve
import decentralized as dec
import pocketknives

π = np.pi
g = 9.80665

In [ ]:
%matplotlib ipympl
plt.rcParams.update({
    "axes.grid": False,
    "figure.constrained_layout.use": True,
    "text.usetex": False,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "ps.distiller.res": 8000,
})

In [ ]:
fig = plt.figure()

## single-agent problem

In [ ]:
dt = 0.05
N = 50
n_d = 2

x = np.array([-10, 10, 10, 0], dtype=float)
x_goal = np.zeros((4, 1), dtype=float).T

# dynamics = dec.DoubleIntDynamics4D(dt)
dynamics = dec.UnicycleDynamics4D(dt)

Q = np.diag([1., 1, 0, 0])
Qf = 1000 * np.eye(Q.shape[0])
R = np.eye(2)
cost = dec.ReferenceCost(x_goal, Q, R, Qf)

prob = dec.ilqrProblem(dynamics, cost)
ilqr = dec.ilqrSolver(prob, N)
X, U, J = ilqr.solve(x)

plt.clf()
plot_solve(X, J, x_goal)
plt.draw()

Show equations of motion, which have the form

$$\dot{x} = f(x, u, p)$$

for states

$$
x = \begin{bmatrix} o_x & o_y & o_z & \psi & \theta & \phi & v_x & v_y & v_z & w_x & w_y & w_z \end{bmatrix},
$$

inputs
$$
u = \begin{bmatrix} \tau_x & \tau_y & \tau_z & f_z \end{bmatrix},
$$

and parameters
$$
p = \begin{bmatrix} m & J_x & J_y & J_z & g \end{bmatrix}.
$$

In [ ]:
# Single-agent problem for a 6D quadcopter (w/ analytical diff.)
dt = 0.1
N = 40
n_d = 3

x = np.array([2, 2, 0.5 , 0, 0, 0], dtype=float)
xf = np.zeros((6, 1), dtype=float).T

dynamics = dec.QuadcopterDynamics6D(dt)

# Q = np.diag([1., 1., 1., 0, 0, 0, 0, 0, 0, 0, 0, 0])
Q = np.diag([1., 1., 1., 1., 1., 1.])
Qf = 100 * np.eye(Q.shape[0])
R = np.diag([0, 1, 1])
cost = dec.ReferenceCost(xf, Q, R, Qf)

prob = dec.ilqrProblem(dynamics, cost)
ilqr = dec.ilqrSolver(prob, N)

# with warnings.catch_warnings():
    # warnings.filterwarnings("error")
X, U, J = ilqr.solve(x)

plt.clf()
plot_solve(X, J, xf, n_d=n_d)
plt.draw()

## multi-agent problem

## 2 quadcopters w/ 1 human 

In [ ]:
def two_quads_one_human():
    x0 = np.array([[-1.5, 0.1, 1, 0, 0, 0,
                     1.5, 0, 1, 0, 0, 0,
                     # 0.0, 0.5, 2, 0, 0, 0,
                    # -2.5, 0.0, 1.5, 0, 0, 0,
                    0, -1, 1.5, 0, 0, 0
                  ]], dtype=float).T
    xf = np.array([[1.5, 0, 2, 0, 0, 0,
                   -1.5, 0, 2, 0, 0, 0,
                   # -2.0, -0.5, 0.5, 0, 0, 0,
                   # 0.2, -1.0, 1.0, 0, 0, 0,
                   0.0, 2, 1.5, 0, 0, 0
                   ]], dtype=float).T
    return x0, xf

In [ ]:
n_agents = 3
n_states = 6
n_controls = 3

x_dims = [n_states] * n_agents
u_dims = [n_controls] * n_agents
n_dims = [3, 3, 2]

dt = 0.05
N = 50
radius = 0.3

x0, xf = two_quads_one_human()

Q = np.diag([1, 1, 1, 5, 5, 5])
R = np.diag([1, 1, 1])
Qf = 1e3 * np.eye(n_states)

Q_human = np.diag([1, 1, 1, 0, 0, 0])
R_human = np.diag([1, 1, 1e-9])
Qf_human = 1e3 * np.eye(Q.shape[0])

Qs = [Q, Q, Q_human]
Rs = [R, R, R_human]
Qfs = [Qf, Qf, Qf_human]

models = [dec.QuadcopterDynamics6D, dec.QuadcopterDynamics6D, dec.HumanDynamics6D]
ids = [100 + i for i in range(n_agents)]
dynamics = dec.MultiDynamicalModel([model(dt, id_) for id_, model in zip(ids, models)])

goal_costs = [
    dec.ReferenceCost(xf_i, Qi, Ri, Qfi, id_)
    for xf_i, id_, x_dim, Qi, Ri, Qfi in zip(
        dec.split_agents_gen(xf, x_dims), ids, x_dims, Qs, Rs, Qfs
    )
]
prox_cost = dec.ProximityCost(x_dims, radius, n_dims)
game_cost = dec.GameCost(goal_costs, prox_cost)

problem = dec.ilqrProblem(dynamics, game_cost)
solver = dec.ilqrSolver(problem, N)

U0 = np.c_[np.tile([g, 0, 0], (N, 2)), np.ones((N, n_controls))]

In [ ]:
X, U, J, _ = dec.solve_centralized(solver, x0, U0, ids, True)

In [ ]:
plt.clf()
plot_solve(X, J, xf, x_dims, True, 3)

In [ ]:
dec.plot_pairwise_distances(X, x_dims, n_dims, radius)

In [ ]:
X0 = np.tile(x0.T, (N+1, 1))
X_dec, U_dec, J_dec, _ = dec.solve_decentralized(problem, X0, U0, radius, verbose=True)

In [ ]:
plt.clf()
plot_solve(X_dec, J_dec, xf, x_dims, True, n_d=3)
plt.title('One-shot decentralized trajectory with Human and Quad')
plt.draw()

In [ ]:
dec.plot_pairwise_distances(X, x_dims, n_dims, radius)

In [ ]:
n_agents = 3
n_states = 6
n_controls = 3

x_dims = [n_states] * n_agents
u_dims = [n_controls] * n_agents
n_dims = [3, 3, 2]

dt = 0.05
N = 50
radius = 0.6

x0, xf = paper_setup_quads_3()

Q = np.diag([1, 1, 1, 5, 5, 5])
R = np.diag([1, 1, 1])
Qf = 1e3 * np.eye(n_states)

Q_human = np.diag([1, 1, 1, 0, 0, 0])
R_human = np.diag([1, 1, 0])
Qf_human = 1e3 * np.eye(Q.shape[0])

Qs = [Q, Q, Q_human]
Rs = [R, R, R_human]
Qfs = [Qf, Qf, Qf_human]

models = [dec.QuadcopterDynamics6D,dec.QuadcopterDynamics6D, dec.HumanDynamics6D]
ids = [100 + i for i in range(n_agents)]
dynamics = dec.MultiDynamicalModel([model(dt, id_) for id_, model in zip(ids, models)])

goal_costs = [
    dec.ReferenceCost(xf_i, Qi, Ri, Qfi, id_)
    for xf_i, id_, x_dim, Qi, Ri, Qfi in zip(
        dec.split_agents_gen(xf, x_dims), ids, x_dims, Qs, Rs, Qfs
    )
]
prox_cost = dec.ProximityCost(x_dims, radius, n_dims)
game_cost = dec.GameCost(goal_costs, prox_cost)

problem = dec.ilqrProblem(dynamics, game_cost)
solver = dec.ilqrSolver(problem, N)

g = 9.80665
U0 = np.tile([g, 0, 0], (N, n_agents))

In [ ]:
X, U, J = solver.solve(x0, U0)

# Multi-agent

In [ ]:
n_states = 4
n_controls = 2
n_agents = 5
x_dims = [n_states] * n_agents
u_dims = [n_controls] * n_agents
n_dims = [2] * n_agents

# ENERGY = 20.0
n_d = n_dims[0]

x0, xf = dec.random_setup(
    n_agents, n_states, 
    is_rotation=False, 
    rel_dist=2.0, 
    var=n_agents/2, 
    n_d=2, 
    random=True
)

x_dims = [n_states] * n_agents
u_dims = [n_controls] * n_agents

# x0 = dec.normalize_energy(x0, x_dims, ENERGY)
# xf = dec.normalize_energy(xf, x_dims, ENERGY)
# x0 = dec.perturb_state(x0, x_dims)

dec.eyeball_scenario(x0, xf, n_agents, n_states)

In [ ]:
dt = 0.05
N = 60

tol = 1e-6
ids = [100 + i for i in range(n_agents)]

model = dec.UnicycleDynamics4D
dynamics = dec.MultiDynamicalModel([model(dt, id_) for id_ in ids])

Q = np.eye(4)
R = np.eye(2)
Qf = 1e3 * np.eye(n_states)
radius = 0.5

goal_costs = [
    dec.ReferenceCost(xf_i, Q.copy(), R.copy(), Qf.copy(), id_)
    for xf_i, id_, x_dim, u_dim in zip(
        dec.split_agents_gen(xf, x_dims), ids, x_dims, u_dims
    )
]
prox_cost = dec.ProximityCost(x_dims, radius, n_dims)
goal_costs = [dec.ReferenceCost(xf_i, Q.copy(), R.copy(), Qf.copy(), id_) 
              for xf_i, id_ in zip(split_agents(xf.T, x_dims), ids)]
prox_cost = dec.ProximityCost(x_dims, radius, n_dims)
game_cost = dec.GameCost(goal_costs, prox_cost)

problem = dec.ilqrProblem(dynamics, game_cost)
solver = dec.ilqrSolver(problem, N)

# U0 = None

In [ ]:
X, U, J = solver.solve(x0, tol=tol, t_kill=None)

In [ ]:
plt.clf()
plot_solve(X, J, xf.T, x_dims, True, n_d)

In [ ]:
plt.clf()
dec.plot_pairwise_distances(X, x_dims, n_dims, radius)

In [ ]:
# (optional) make gif.
dec.make_trajectory_gif("traj.gif", X, xf.ravel(), x_dims, radius)